In [1]:
!pip install langchain-google-genai langchain pypdf faiss-cpu langchain_community

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp312-cp312-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.4-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp312-cp312-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set the Google API key from environment variables
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [3]:
# Load the PDF file
loader = PyPDFLoader("Human-Resource-Management.pdf")
documents = loader.load()

In [8]:
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [9]:
# Create embeddings and FAISS vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [10]:
# Define the prompt template
prompt_template = """
Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n

Answer:
"""

# Initialize the Gemini Pro model
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)

# Create the prompt from the template
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Create the QA chain
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [11]:
# Ask a question
question = "What are the key areas of HR management?"
docs = vector_store.similarity_search(question)
response = chain({"input_documents": docs, "question": question}, return_only_outputs=True)
print(response['output_text'])

Based on the provided text, the key areas of HR management are:

1. **Personnel Aspect:** This encompasses manpower planning, recruitment, selection, placement, induction, transfer, promotion, demotion, termination, training & development, layoff & retrenchment, wage & salary administration, incentives, and productivity.

2. **Welfare Aspect:** This focuses on working conditions and amenities such as canteens, crèches, rest rooms, lunch rooms, housing, transport, education, medical help, health & safety, washing facilities, and recreation & cultural activities.

3. **Industrial Aspect:** This deals with employee relations, including union management relations, joint consultation, negotiating collective bargaining, grievance handling, disciplinary actions, and the settlement of industrial disputes.  The text also mentions industrial relations as a key area, encompassing relations among employees, employers, government, and trade unions (including trade unions, collective bargaining, and